# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

df = pd.read_csv('zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [25]:
#Selecting data from Austin
df_atx = df[df['City'] == 'Austin']

In [26]:
df_atx.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
66,92617,78704,Austin,TX,Austin,Travis,67,221300.0,221100.0,221000.0,...,530100,532400,534200,536200,538200,539800,540200,542300,547100,550900
98,92654,78745,Austin,TX,Austin,Travis,99,135000.0,134200.0,133800.0,...,286300,286900,287500,288700,289800,290500,291300,292600,294000,294700
422,92667,78758,Austin,TX,Austin,Travis,423,129000.0,128300.0,127500.0,...,250000,248400,247500,247400,247900,250500,255100,260100,266000,270400
432,92651,78741,Austin,TX,Austin,Travis,433,93800.0,93600.0,93500.0,...,228000,226700,227600,230200,234100,238300,241000,243600,247400,250400
502,92662,78753,Austin,TX,Austin,Travis,503,111300.0,110600.0,109900.0,...,222600,223200,223600,223100,223100,225000,227100,228700,231500,233400


In [28]:
df_atx

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
66,92617,78704,Austin,TX,Austin,Travis,67,221300.0,221100.0,221000.0,...,530100,532400,534200,536200,538200,539800,540200,542300,547100,550900
98,92654,78745,Austin,TX,Austin,Travis,99,135000.0,134200.0,133800.0,...,286300,286900,287500,288700,289800,290500,291300,292600,294000,294700
422,92667,78758,Austin,TX,Austin,Travis,423,129000.0,128300.0,127500.0,...,250000,248400,247500,247400,247900,250500,255100,260100,266000,270400
432,92651,78741,Austin,TX,Austin,Travis,433,93800.0,93600.0,93500.0,...,228000,226700,227600,230200,234100,238300,241000,243600,247400,250400
502,92662,78753,Austin,TX,Austin,Travis,503,111300.0,110600.0,109900.0,...,222600,223200,223600,223100,223100,225000,227100,228700,231500,233400
507,92668,78759,Austin,TX,Austin,Travis,508,266400.0,269000.0,272000.0,...,429700,431100,433000,435800,439400,442400,443200,445000,449900,454100
594,92657,78748,Austin,TX,Austin,Travis,595,199500.0,198800.0,197900.0,...,268400,268700,270100,272600,274900,276700,277800,279000,280800,282000
1671,92653,78744,Austin,TX,Austin,Travis,1672,85400.0,85400.0,85300.0,...,180300,182900,186000,188400,189700,190900,192100,193200,194500,195500
1936,92658,78749,Austin,TX,Austin,Travis,1937,264000.0,266400.0,269300.0,...,350900,350400,349400,348900,348700,348900,349700,352100,355500,357700
2462,92634,78723,Austin,TX,Austin,Travis,2463,97600.0,99000.0,100300.0,...,336300,339000,340500,343100,347800,352500,355800,358700,361900,364200


In [29]:
#Select Austin Texas
df_atx = df_atx[df_atx['State'] == 'TX']

In [30]:
df_atx

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
66,92617,78704,Austin,TX,Austin,Travis,67,221300.0,221100.0,221000.0,...,530100,532400,534200,536200,538200,539800,540200,542300,547100,550900
98,92654,78745,Austin,TX,Austin,Travis,99,135000.0,134200.0,133800.0,...,286300,286900,287500,288700,289800,290500,291300,292600,294000,294700
422,92667,78758,Austin,TX,Austin,Travis,423,129000.0,128300.0,127500.0,...,250000,248400,247500,247400,247900,250500,255100,260100,266000,270400
432,92651,78741,Austin,TX,Austin,Travis,433,93800.0,93600.0,93500.0,...,228000,226700,227600,230200,234100,238300,241000,243600,247400,250400
502,92662,78753,Austin,TX,Austin,Travis,503,111300.0,110600.0,109900.0,...,222600,223200,223600,223100,223100,225000,227100,228700,231500,233400
507,92668,78759,Austin,TX,Austin,Travis,508,266400.0,269000.0,272000.0,...,429700,431100,433000,435800,439400,442400,443200,445000,449900,454100
594,92657,78748,Austin,TX,Austin,Travis,595,199500.0,198800.0,197900.0,...,268400,268700,270100,272600,274900,276700,277800,279000,280800,282000
1671,92653,78744,Austin,TX,Austin,Travis,1672,85400.0,85400.0,85300.0,...,180300,182900,186000,188400,189700,190900,192100,193200,194500,195500
1936,92658,78749,Austin,TX,Austin,Travis,1937,264000.0,266400.0,269300.0,...,350900,350400,349400,348900,348700,348900,349700,352100,355500,357700
2462,92634,78723,Austin,TX,Austin,Travis,2463,97600.0,99000.0,100300.0,...,336300,339000,340500,343100,347800,352500,355800,358700,361900,364200


# Step 2: Data Preprocessing

In [2]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [3]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [4]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [5]:
df.isna().sum()

RegionID         0
RegionName       0
City             0
State            0
Metro         1043
              ... 
2017-12          0
2018-01          0
2018-02          0
2018-03          0
2018-04          0
Length: 272, dtype: int64

In [24]:
df['Metro'].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
14718    False
14719    False
14720    False
14721     True
14722    False
Name: Metro, Length: 14723, dtype: bool

In [14]:
no_metro = df['Metro'] == 'Nan'

In [15]:
df[no_metro]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04


# Step 3: EDA and Visualization

In [29]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)
# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [31]:
plt.gcf().autofmt_xdate()
plt.show()

<Figure size 432x288 with 0 Axes>

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results